# Atomic relaxation data

Atomic relaxation data is used to calculate the X-ray and Auger electron spectrum that arises from an ionised atom relaxing back to a lower energy state. Atoms can be ionised through various mechanisms, including but not limited to atomic interactions of photons (photoatomic ionisation), electrons (electroatomic ionisation), etc. When an atom is ionised, a vacancy in one of the atom's electron subshells is created. These vacancies can be filled by the transition of an electron from a higher subshell (which will have a lower binding energy than the subshell with the vacancy). This transition can either be radiative or non-radiative.

In a radiative transition, an electron from another shell fills a vacancy in the current shell while emitting a photon (this electron is often referred to as an X-ray). In a non-radiative transition, an electron from another shell fills a vacancy in the current shell while another electron is emitted (this electron is often referred to as an Auger electron).

Atomic relaxation data in `dryad` is stored within an `AtomicRelaxation` object. The underlying objects inside of an `AtomicRelaxation` object are described here:
- [Transition data](atomic-relaxation-data/1-transition-data.ipynb): `RadiativeTransitionData` and `NonRadiativeTransitionData` to store transition data (shell identifiers, probabilities and optional transition energies)
- [Electron subshell configuration data](atomic-relaxation-data/2-subshell-configuration-data.ipynb): `ElectronSubshellConfiguration` to store transition data and other information for each electron subshell of an atom in the atomic relaxation library

## `AtomicRelaxation`

### Reading atomic relaxation data from a file

An `AtomicRelaxation` instance can currently be created from a ENDF6 or GNDS formatted data files. Reading this type of data from an ACE file will be added at a later time. A number of dedicated static factory functions for each supported format are defined in the `AtomicRelaxation` interface:
- `from_endf_file` for ENDF6 formatted files
- `from_gnds_file` for GNDS 2.0 formatted xml files

The arguements to be provided to these functions are:
- `filename`: the name and path (relative or absolute) of the file
- `normalise`: an option to indicate whether or not to normalise all probability data (default: no normalisation)

Note: the default behaviour in any `dryad` object that has probability data is that a user must explicitly request normalisation at construction or read time.

For example:

In [1]:
from dryad import AtomicRelaxation

oxygen_endf = AtomicRelaxation.from_endf_file( 'resources/atomic-relaxation-data/atom-008_O_000.endf' )
oxygen_gnds = AtomicRelaxation.from_gnds_file( 'resources/atomic-relaxation-data/atom-008_O_000.endf.gnds.xml' )

### Properties

`AtomicRelaxation` contains the atomic relaxation data for a single atom:
- the element identifier for the atom for which the atomci relaxation data is given (`element_identifier`)
- the data for each subshell of the atom (`subshells`)

The `element_identifier` property is read-only while the `subshells` property is a read and write property.

The subshell data is sorted by the order given by the electron subshell identifier (i.e. first the K shell, then the L1 shell, etc.).

The `AtomicRelaxation` interface also provides a `has_subshell(id)` function to check whether or not a given electron subshell is present and a `subshell(id)` function that can be used to extract a specific subshell (an exception is thrown if the subshell cannot be found).

### Construction

To create an instance of `AtomicRelaxation` the following arguments are required:
- `element`: the element identifier of the atom
- `subshells`: a list of `ElectronSubshellConfiguration` instances with the data for each electron subshell of the atom

and the following argument is optional:
- `normalise`: an option to indicate whether or not to normalise all probability data (default: no normalisation)

### Calculate transition energies

Since the transition energy can be calculated using the binding energy of the current subshell and the originating subshell (see [Transition data](atomic-relaxation-data/1-transition-data.ipynb) for more information), the transition energy is considered a redundant quantity. As such, atomic relaxation data read from a GNDS file does not populate this quantity. These can be calculated by calling the `calculate_transition_energies()` function on an `AtomicRelaxation` instance.

### Normalisation of transition probabilities

As indicated above, when constructing or reading atomic relaxation data, the user has the possibility of normalising the probability data (the `normalise` option) so that the sum of all transition probabilities for each electron subshell is 1 (if the subshell has transition data). The `normalise()` allows a user to do the same after the `AtomicRelaxation` instance was created.

If the user changes one or more transition probabilities, this function can then be used to renormalise the data. It is the responsibility of the user to request this renormalisation whenever required.